In [ ]:
!pip install -r requirements.txt  &> /dev/null
%rm -r dreamgaussian
!git clone https://github.com/dreamgaussian/dreamgaussian
!pip install -q einops plyfile dearpygui huggingface_hub diffusers accelerate transformers xatlas trimesh PyMCubes pymeshlab rembg[gpu,cli] omegaconf ninja
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q https://github.com/camenduru/diff-gaussian-rasterization/releases/download/v1.0/simple_knn-0.0.0-cp310-cp310-linux_x86_64.1.whl
!pip install -q git+https://github.com/ashawkey/kiuikit
!pip install -q git+https://github.com/NVlabs/nvdiffrast


Cloning into 'dreamgaussian'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 120 (delta 30), reused 21 (delta 19), pack-reused 74
Receiving objects: 100% (120/120), 1.23 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (49/49), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
import production_ready
import os
from IPython.display import Image, display
from IPython.display import HTML
from base64 import b64encode

In [ ]:
config = production_ready.prompt_user()

Please enter a prompt: cone


In [ ]:
log_dir = production_ready.generate_images(config)

wandb: Currently logged in as: rahulmark42. Use `wandb login --relogin` to force relogin


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
100%|██████████| 50/50 [00:43<00:00,  1.14it/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transfor

Evaluation results tensor(5.0547, device='cuda:0', dtype=torch.float16)
cone
cone
cone
cone
cone
cone
cone
cone


In [ ]:
def dream_gaussian_pipeline(log_dir):
  file_list = os.listdir(log_dir)
  for selected_file in file_list:
    file_to_process = log_dir + '/' + selected_file
    %run dreamgaussian/process.py {file_to_process}
    NAME = selected_file[:-4]
    IMAGE_PROCESSED = NAME + '_rgba.png'
    Elevation = 0
    %run dreamgaussian/main.py --config dreamgaussian/configs/image.yaml input={log_dir + '/' + IMAGE_PROCESSED} save_path={NAME} elevation={Elevation} force_cuda_rast=True
    %run dreamgaussian/main2.py --config dreamgaussian/configs/image.yaml input={log_dir + '/' + IMAGE_PROCESSED} save_path={NAME} elevation={Elevation} force_cuda_rast=True

In [ ]:
%run -m kiui.render {log_dir  + '/' + NAME}.obj --save_video {log_dir + '/' +  NAME}.mp4 --wogui --force_cuda_rast
def show_video(video_path, video_width=450):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
#show_video(f'{log_dir}/{selected_file}/{NAME}.mp4')